In [1]:
from runner import *

In [2]:
dests,routes,ascountry,country,rings,costs,sites=loadInits()


Init Begin
sites Loaded
Rings Loaded
Costs Loaded
1.0
Routes Loaded
Dests are seted num Dests 16841


In [3]:
sources=ascountry
excludes= rings
routes.routes['10212']['270'].alternativepaths


[([10212, 4808, 4837, 209, 297, 270], '60'),
 ([10212, 4812, 4134, 209, 297, 270], '60'),
 ([10212, 4813, 4134, 209, 297, 270], '60'),
 ([10212, 4847, 4134, 209, 297, 270], '60'),
 ([10212, 17621, 4837, 209, 297, 270], '60'),
 ([10212, 18118, 4847, 4134, 209, 297, 270], '60')]

In [13]:
import myDB
import operator
import matplotlib.pyplot as plt
import random
import game

import cPickle as pickle
import math
import signal
import numpy as np


def handler(signal, frame):
    print 'signal'
    raise ArithmeticError()


def compareRAD(r1, r2):
    per1 = int(r1[1])
    per2 = int(r2[1])
    # LOCAL PRE
    if per2 > per1:
        return r2
    elif per1 > per2:
        return r1

    # SHORTEST
    if len(r2[0]) > len(r1[0]):
        return r1
    if len(r2[0]) < len(r1[0]):
        return r2
    if r1[0][0] < r2[0][0]:
        return r1
    return r2


def select_random(dests, country, cost, properties, min_cone, excludes):
    dec = []
    proten = []
    for i in dests:
        sel = i[0]
        if properties[str(sel)]['Country'] != country and int(
                str(properties[str(sel)]['#AS']).replace(',', '')) >= min_cone and not excludes.has_key(str(sel)):
            proten.append(sel)
    np.random.shuffle(proten)
    sel_cost = 0
    sels = []
    tryies = 1000
    for i in proten:
        i = str(i)
        i_cost = game.deployCostFucntion (properties[i], country)
        if i_cost + sel_cost <= cost:
            sels.append(i)
            sel_cost += i_cost
        if sel_cost > cost:
            tryies -= 1
            if tryies < 0:
                return sels
    return sels


def select_aggresive(dest, country, cost, properties, excludes):
    i = 0
    dec = []
    tries = 1000
    sel_cost = 0
    while i < len(dest):
        if properties[str(dest[i][0])]['Country'] != country and not excludes.has_key(str(dest[i][0])):
            q = str(dest[i][0])
            i_cost = game.deployCostFucntion(properties[q], country)
            if i_cost + sel_cost <= cost:
                dec.append(q)
                sel_cost += i_cost
            if sel_cost > cost:
                tries -= 1
                if tries < 0:
                    return dec

        i += 1
    return dec


def evalute_options(dests, sources, soc, properties):
    pass


def computeImportance(routes, sources, costs):
    contents = {}
    ASes = {}
    Revs = {}
    counts=0
    for i in sources:
        if not routes.has_key(i):
            continue
        dests = routes[i]
        for d in dests:
            if costs[d]['Country'] == 'US':
                ASes.setdefault(d,{})

                if len(routes[i][d].selectedpath)<1:
                    continue
                for ass in routes[i][d].selectedpath[0]:
                    #print ass,routes[i][d].selectedpath
                    ASes[d].setdefault(ass, []).append(i)
                    Revs.setdefault(str(ass), {}).setdefault(str(d), []).append(str(i))
                '''for route in routes[i][d].alternativepaths:
                    for ass in route[0]:

                        ASes[d].setdefault(ass, []).append(i)
                        Revs.setdefault(str(ass), {}).setdefault(str(d), []).append(str(i))'''

    print len(Revs)
    return Revs


def decoyGameAction(routers, sources, country, properties, budget, excludes):
    #dec= pickle.load(open('game_2e7.pickle'))
    #return dec

    imps=computeImportance(routers,sources,properties)

    print 'decoy: IMPS COMPUTED'
    dec=game.decoyResonse(imps.keys(),country,properties,budget,excludes,imps)
    #pickle.dump(dec, open('ress/game_naughty2_%d.pickle'%budget,'wb'))
    print 'decoy: decoys deployed'
    return dec


def compute(dests, routes, ascountry, country, sources, percentage):
    results = {}
    sorted_dsts = dests
    selection_method_str = 'random'
    costs = pickle.load(open('../Datasets/AS_Cost.pickle', 'rb'))
    soc = int(len(sorted_dsts) * float(percentage) / 100.0)
    dec = select_random(sorted_dsts, country, soc, costs, 0)
    results['Decoys'] = dec

    rebuild = {}
    needrebuild = {}

    oks = {}
    print dec
    com = 0
    print 'start for %f percent with %d seleced' % (percentage, soc)
    lastDests = {}
    for i in sources:
        paths = routes.getRouteForSource(i)
        com += 1
        print 'computeing %d of %d' % (com, len(sources))
        # print len(paths)
        for k in paths:
            if k in sources:
                continue

            if lastDests.get(k, 0) == -2:
                # print 'escape'
                continue
            # print k
            route, type = routes.computeRAD(i, k, dec, sources)

            lastDests[k] = type

            if type >= 0:
                rebuild.setdefault(k, []).append(route)

                oks[k] = 1
            else:
                needrebuild.setdefault(i, []).append(k)

            # print route,type
            results.setdefault(i, {})[k] = {'route': route, 'type': type}

    # print oks
    best_paths = {}

    for k in rebuild:
        best = rebuild[k][0]
        for p in rebuild[k]:
            best = compareRAD(best, p)
        best_paths[k] = best

    for i in needrebuild:
        for k in needrebuild[i]:
            if not rebuild.has_key(k):
                results.setdefault(i, {})[k] = {'route': best, 'type': -1}

                continue
            best = best_paths[k]

            results.setdefault(i, {})[k] = {'route': best, 'type': -2}

    results['unreachable'] = float(len(oks)) / float(len(dests))

    pickle.dump(results, open('../results/results-%f-%s.pickle' % (percentage, selection_method_str), 'wb'))
    print 'done percentage %f' % percentage, float(len(oks)) / float(len(dests))
    return



def censorGameAction(routes,sources,country,costs,decoys,sites,routemanager):
    game.computeAllReroute(sources,routemanager,decoys)
    dic={}

    for source in routes:
        for dest in routes[source]:
            game.gameCensorDecide(source,dest,costs,sites,routemanager)


def analysResult(routes,costs):
    results={'NVF':[],'Unreachable':[],'Decoy':[],'Less Preferred':[],'Not changed':[]}
    decoyutility=0
    censorutility=0


    for s in routes:
        for d in routes[s] :
            if routes[s][d].isBGP:
                if routes[s][d].routetype!=0:
                    results['Decoy'].append((s,d))
                    decoyutility+=( float(str(costs[s].get('#IP','0')).replace(',','')) )*( float(str(costs[d].get('#IP','0')).replace(',','')) )
                else:
                    results['Not changed'].append((s,d))

            else:
                if routes[s][d].routetype==1:
                    results['Less Preferred'].append((s,d))
                if routes[s][d].routetype==2:
                    results['NVF'].append((s,d))
                if routes[s][d].routetype==-1:
                    results['Unreachable'].append((s,d))
    colors=['yellowgreen','gold','lightskyblue','lightcoral','gray']
    #print results
    plt.ioff()
    plt.pie([len(results[i]) for i in results],labels=[i for i in results],colors=colors,shadow=True,startangle=90,explode=(0,0,0.1,0,0),autopct='%1.1f%%')
    plt.show()

    x= {i:len(results[i]) for i in results}
    x['decoyutility']=decoyutility
    print x



def hasDecoy(r,decoys,debug=False):
        for i in r[0]:
            if decoys.has_key(str(i)):
                if debug:
                    print i,
                return True
        return False


def computeGame():
    pass
def computeDecoyUtility(routes,decoys,costs,debug,rm):
    utility =0
    for s in routes:
        for d in routes[s] :
            if hasDecoy( routes[s][d].selectedpath,decoys):
                if routes[s][d].routetype==0 and debug:
                    print routes[s][d].selectedpath ,routes[s][d].routetype , routes[s][d].isBGP,( float(str(costs[s].get('#IP','0')).replace(',','')) )*( float(str(costs[d].get('#IP','0')).replace(',','')) )
                    hasDecoy(routes[s][d].selectedpath,decoys,True)
                    print rm.computeRAD(s, d, decoys)
                    print rm.hasDecoy(routes[s][d].selectedpath,decoys)
                    print hasDecoy(routes[s][d].bestpath,decoys,True)

                utility+=( float(str(costs[s].get('#IP','0')).replace(',','')) )*( float(str(costs[d].get('#IP','0')).replace(',','')) )
    return utility
def experiment(routes,dests, sources,country, excludes,costs,sites):
    reload(game)
    sources=sources['Country2ASN'][country]
    decoy_max_costs=1
    roundcount=0
    decutility=[]
    while True:

        if roundcount>-1:
            dec=decoyGameAction(routes.routes,sources,country,costs,decoy_max_costs,excludes)

        roundcount+=1
        print 'Decoy: Computed '
        ut=computeDecoyUtility(routes.routes,dec,costs,False,routes)
        decutility.append(ut)
        print 'Decoy: Computed DECOY utility= ',ut
        censorGameAction(routes.routes,sources,country,costs,dec,sites,routes)
        ut=computeDecoyUtility(routes.routes,dec,costs,True,routes)
        decutility.append(ut)
        print 'Censor: Computed decoy utility= ',ut

        analysResult(routes.routes,costs)

In [5]:
reload(game)
sources=sources['Country2ASN'][country]
decoy_max_costs=1
roundcount=0
decutility=[]

In [6]:
dec=decoyGameAction(routes.routes,sources,country,costs,decoy_max_costs,excludes)


332
decoy: IMPS COMPUTED
332 332
Sorted
['15576']
decoy: decoys deployed


AttributeError: Route instance has no attribute 'isBGP'

In [17]:
%debug censorGameAction(routes.routes,sources,country,costs,dec,sites,routes)

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
None
> <string>(1)<module>()

ipdb> s
--Call--
> <ipython-input-13-92334749508b>(200)censorGameAction()
    198 
    199 
--> 200 def censorGameAction(routes,sources,country,costs,decoys,sites,routemanager):
    201     game.computeAllReroute(sources,routemanager,decoys)
    202     dic={}

ipdb> s
> <ipython-input-13-92334749508b>(201)censorGameAction()
    199 
    200 def censorGameAction(routes,sources,country,costs,decoys,sites,routemanager):
--> 201     game.computeAllReroute(sources,routemanager,decoys)
    202     dic={}
    203 

ipdb> r
Computing RAD
Censor: RAD COMPUTED
--Return--
None
> <ipython-input-13-92334749508b>(204)censorGameAction()
    202     dic={}
    203 
--> 204     for source in routes:
    205         for dest in routes[source]:
    206             game.gameCensorDecide(source,dest,costs,sites,routemanager)

ipdb> b 206
Breakpoint 1 at <ipython-input-13-92334749508b>:206
ipdb> r
--Return--
None
> <s

In [11]:
analysResult(routes.routes,costs)

{'Not changed': 276, 'Decoy': 0, 'decoyutility': 0, 'NVF': 0, 'Unreachable': 0, 'Less Preferred': 0}
